In [1]:
from aocd import data, models, submit
from io import StringIO
from pathlib import Path
import re

import pandas as pd

# Load data and examples

In [2]:
puzzle_year = 2024
puzzle_day = int(re.match(r"day(\d+)", Path.cwd().name).group(1))

In [3]:
todays_puzzle = models.Puzzle(year=puzzle_year, day=puzzle_day)
todays_examples = todays_puzzle.examples
data = todays_puzzle.input_data

In [ ]:
todays_examples[0] = todays_examples[0]._replace(
    input_data="""1
10
100
2024"""
)
todays_examples

# Part A

In [5]:
def mix(value, secret):
    return value ^ secret


def prune(value):
    return value % 16777216


def calculate_next_secret(secret):
    new_secret = prune(mix(secret * 64, secret))
    new_secret = prune(mix(new_secret // 32, new_secret))
    new_secret = prune(mix(new_secret * 2048, new_secret))
    return new_secret


def iterate_secret_n_times(secret, n):
    for i in range(n):
        secret = calculate_next_secret(secret)
    return secret

In [6]:
def part_a(data: str) -> str:
    secrets = [int(x) for x in data.split("\n")]
    result = 0
    for secret in secrets:
        result += iterate_secret_n_times(secret, 2000)
    return str(result)

In [ ]:
for example_index, example in enumerate(todays_examples):
    if example.answer_a != "":
        print(
            f"Example {example_index} part a: {part_a(example.input_data)} (expected {example.answer_a})"
        )
        assert part_a(str(example.input_data)) == example.answer_a
submit(part_a(data), part="a", year=puzzle_year, day=puzzle_day)

# Part B

In [271]:
from itertools import product
from tqdm import tqdm
import numpy as np

In [268]:
def get_buyers_prices(secret):
    prices = [secret % 10]
    for i in range(2000):
        secret = calculate_next_secret(secret)
        prices.append(secret % 10)
    return prices


def get_diff_sequence_strings_and_price(prices) -> dict[str, int]:
    buyers_diff = np.diff(prices)

    result_dict = {}
    for i in range(len(buyers_diff) - 3):
        sequence = ",".join(buyers_diff[i : i + 4].astype(str))
        if sequence not in result_dict:
            # accept only first appearance of a sequence
            result_dict[sequence] = int(prices[i + 4])
    return result_dict

In [273]:
def part_b(data: str) -> str:
    secrets = [int(x) for x in data.split("\n")]
    buyers_diff_sequence_strings_and_prices = []
    for secret in tqdm(secrets):
        prices = get_buyers_prices(secret)
        buyers_diff_sequence_strings_and_prices.append(
            get_diff_sequence_strings_and_price(prices)
        )
    combined_sequences = {}
    for x in buyers_diff_sequence_strings_and_prices:
        for key in x.keys():
            if key not in combined_sequences:
                combined_sequences[key] = 0
            combined_sequences[key] += x[key]
    max_price = 0
    for key in combined_sequences.keys():
        if combined_sequences[key] > max_price:
            max_price = combined_sequences[key]
    return str(max_price)

In [ ]:
assert part_b("1\n2\n3\n2024") == "23"
submit(part_b(data), part="b", year=puzzle_year, day=puzzle_day)